## <font color = "#CC3D3D">Case #1: Background Subtraction</font>

<hr>

### Step #1: Concept
<hr>

- 배경 차분(Background Subtraction: BS)
    - 등록된 배경 모델과 현재 입력 프레임과의 차영상을 이용하여 전경 객체를 검출하는 기법이다.
    - 움직이는 전경 객체 검출을 위한 기본적인 방법이다.
    - <img src="images/markdown/.png" width="600">

<hr>

### Step #2: Static Background Subtraction
<hr>

In [21]:
# 정적 배경을 이용한 전경 객체 검출 예제
import sys

import cv2

capture = cv2.VideoCapture("videos/PETS2000.avi")

if not capture.isOpened():
    print("Video open failed!")
    sys.exit()
else:
    print("Video open succeed!")

Video open succeed!


In [22]:
# 첫 번째 프레임을 배경 영상으로 등록
retval, background = capture.read()

if not retval:
    print("Background image registration failed!")
    sys.exit()
else:
    print("Background image registration succeed!")

# 컬러 값이 반드시 필요한 상황이 아니고, 연산 속도를 증가시키기 위해 그레이스케일로 변환
background_gray = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
background_gray = cv2.GaussianBlur(background_gray, (0, 0), 1)

Background image registration succeed!


In [23]:
# 비디오 매 프레임 처리
while True:
    retval, frame = capture.read()
    
    if not retval:
        break
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.GaussianBlur(frame_gray, (0, 0), 1)
    
    # 차영상 구하기 & 이진화
    difference = cv2.absdiff(background_gray, frame_gray)
    _, difference = cv2.threshold(difference, 30, 255, cv2.THRESH_BINARY)
    
    # 레이블링 기법을 이용하여 원본 프레임에 바운딩 박스 표시
    count, _, stats, _ = cv2.connectedComponentsWithStats(difference)
    
    for i in range(1, count):
        x, y, w, h, s = stats[i]
        
        if s < 100:
            continue
        
        cv2.rectangle(frame, (x, y, w, h), (0, 0, 255), 2)
    
    # OpenCV 가상 윈도우로 출력
    cv2.imshow("Frame", frame)
    cv2.imshow("Difference", difference)

    if cv2.waitKey(30) == 27:
        print("Video was interrupted!")
        break

capture.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

Video was interrupted!


-1

<hr>

### Step #3: Moving Average Background Subtraction
<hr>

- 정적 배경 모델 사용 시 문제점
    - 미리 등록된 디준 영상이 실제 배경과 크게 달라질 경우 오동작을 일으킬 수 있다.
        - e.g., 그림자 등의 영향으로 인한 `조도 변경`, 시점 변경으로 인한 `밝기 감소`, `새로운 객체가 화면에 고정`되는 경우
        - <img src="images/markdown/.png" width="600">

<hr>

- 평균 연산에 의한 배경 영상 생성
    - 움직이는 객체가 존재하는 수백 장의 입력 영상으로부터 `평균 영상`을 구한다.
    - <img src="images/markdown/.png" width="600">
    - 다만 수백 장의 이전 프레임을 버퍼에 저장하려면 `대용량 메모리가 필요`하다.

<hr>

- 이동 평균(Moving Average)
    - 수백 장의 영상을 저장하는 대신 `매 프레임이 들어올 때마다 평균 영상을 갱신`한다.
    - 평균 연산에 비해 대용량 버퍼 메모리가 필요하지 않다.
    - 내부 연산은 `cv2.addWeighted()` 함수와 같은 `가중치 합`을 구하는 형태이며, 이전 프레임까지의 `배경 영상`과 `현재 프레임`의<br>
    `가중치 합`을 계산하여 현재 프레임에서의 `배경 영상`을 업데이트 하는 방법이다.

$$B(x,y,t)=\alpha\cdot I(x,y,t)+(1-\alpha)\cdot B(x,y,t-1)$$
$$B(x,y,t)={\scriptstyle\text{갱신된 배경 영상}}$$
$$\alpha={\scriptstyle\text{현재 프레임에 대한 가중치}},\ (0<\alpha<1)$$
$$I(x,y,t)={\scriptstyle\text{현재 프레임}}$$
$$B(x,y,t-1)={\scriptstyle\text{이전 프레임까지의 배경 영상}}$$

<hr>

### Step #3-1: OpenCV function
<hr>

> `이동 평균 계산을 위한 가중치 누적 함수`

$$\mathsf{{\color{RoyalBlue}cv2.}{\color{Tan}accumulateWeighted}(src, dst, alpha, mask)\rightarrow }$$
- src: 입력 영상
- dst: 축적 영상(결과 영상)
- alpha: (입력 영상에 대한) 가중치
- mask: 마스크 연산 시 사용할 마스크 영상
- `참고사항:`
    - src: 1 or 3 channel, 8 bit or 32 bit float type
    - dst: src와 동일 채널, 32 bit or 64 bit float type
    - alpha: `1을 지정`하면 이전까지의 배경 영상을 무시하고 `현재 프레임을 배경 영상으로 사용`한다.<br>
    반대로 `0을 지정`하면 현재 프레임에 대한 정보가 사라지고 처음에 모델로 사용한 배경 영상을 계속 사용한다.`(정적 배경 모델)`
        - 일반적으로 `0에 가까운 값`을 지정한다.`(e.g., alpha = 0.01 or less)`

<hr>

### Step #3-2: Implementation example
<hr>

In [55]:
# 이동 평균에 의한 배경 차분 예제
import sys

import cv2
import numpy as np

capture = cv2.VideoCapture("videos/PETS2000.avi")

if not capture.isOpened():
    print("Video open failed!")
    sys.exit()
else:
    print("Video open succeed!")

Video open succeed!


In [56]:
# 첫 번째 프레임을 배경 영상으로 등록
retval, background = capture.read()

if not retval:
    print("Background image registration failed!")
    sys.exit()
else:
    print("Background image registration succeed!")

# background: uint8 배경, background_float: float32 배경
background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
background = cv2.GaussianBlur(background, (0, 0), 1)
background_float = background.astype(np.float32)

Background image registration succeed!


In [57]:
# 비디오 매 프레임 처리
while True:
    retval, frame = capture.read()
    
    if not retval:
        break
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.GaussianBlur(frame_gray, (0, 0), 1)
    
    # background_float: 이전 프레임까지의 배경(float32), background: 갱신된 배경(uint8)
    cv2.accumulateWeighted(frame_gray, background_float, 0.01)
    background = background_float.astype(np.uint8)
        
    # 차영상 구하기 & 이진화
    difference = cv2.absdiff(background, frame_gray)
    _, difference = cv2.threshold(difference, 30, 255, cv2.THRESH_BINARY)
    
    # 레이블링 기법을 이용한 원본 프레임의 움직이는 객체에 바운딩 박스 표시
    count, _, stats, _ = cv2.connectedComponentsWithStats(difference)
    
    for i in range(1, count):
        x, y, w, h, s = stats[i]
        
        if s < 100:
            continue
        
        cv2.rectangle(frame, (x, y, w, h), (0, 0, 255), 2)
    
    # OpenCV 가상 윈도우로 출력
    cv2.imshow("Frame", frame)
    cv2.imshow("Difference", difference)
    cv2.imshow("Background", background)

    if cv2.waitKey(30) == 27:
        print("Video was interrupted!")
        break

capture.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

Video was interrupted!


-1

<hr>

### Step #4: MOG Background Model
<hr>

- MOG
    - Mixture of Gaussian, GMM(Gaussian Mixture Model)
    - 각 픽셀에 대해 MOG 확률 모델을 설정하여 배경과 전경을 구분하는 일종의 데이터 분석 기법이다.

<hr>

- 다양한 배경 모델 구성 방법
    - <img src="images/markdown/.png" width="600">

<hr>

- dfdfdfd
    - <img src="images/markdown/.png" width="600">

<hr>

- dfdfdf
    - <img src="images/markdown/.png" width="600">

<hr>

### Step #4-1: OpenCV function
<hr>

> `배경 차분 모델 객체 생성`

$$\mathsf{{\color{RoyalBlue}cv2.}{\color{Tan}createBackgroundSubtractorMOG2}(, history, varThreshold, detectShadows)\rightarrow retval}$$
- history: 히스토리 길이
- varThreshold: variance threshold. 픽셀과 모델 사이의 마할라노비스 거리(Mahalanobis distance) 제곱에 대한 임계값
- detectShadows: 그림자 검출 여부
- retval: cv2.BackgroundSubtractor 클래스 인스턴스(객체)
- `참고사항:`
    - history: 이전(과거) 프레임을 몇 개까지 사용할지를 지정한다.
        - `기본값은 500`
    - varThreshold: 해당 픽셀이 배경 모델에 의해 잘 표현되는 지를 판단한다. 이 파라미터는 배경 영상 갱신에 영향을 주지 않는다.
        - `기본값은 16`
    - detectShadows: `기본값은 True`

$$\mathsf{{\color{RoyalBlue}cv2.}{\color{Tan}createBackgroundSubtractorKNN}(, history, dist2Threshold, detectShadows)\rightarrow }$$
- history: 
- dist2Threshold: distance to threshold. 픽셀과 샘플 사이의 거리 제곱에 대한 임계값
- detectShadows: 
- retval: cv2.BackgroundSubtractor 클래스 인스턴스(객체)
- `참고사항:`
    - history: 이전(과거) 프레임을 몇 개까지 사용할지를 지정한다.
        - `기본값은 500`
    - dist2Threshold: 해당 픽셀이 샘플에 가까운 지를 판단한다. 이 파라미터는 배경 영상 갱신에 영향을 주지 않는다.
        - `기본값은 400`
    - detectShadows: `기본값은 True`

<hr>

> `전경 객체 마스크 생성 함수`

$$\mathsf{{\color{RoyalBlue} }{\color{Tan} }()\rightarrow }$$

<hr>

### Step #4-2: Implementation example
<hr>

In [ ]:
help(cv2.createBackgroundSubtractorMOG2)

In [ ]:
help(cv2.createBackgroundSubtractorKNN)

## <font color = "#CC3D3D">Case #2: Background Subtraction</font>

In [ ]:
$$\mathsf{{\color{RoyalBlue} }{\color{Tan} }()\rightarrow }$$
- <img src="images/markdown/.png" width="600">

<hr>

### Tracing vs Tracking
<hr>

- To trace: follow the completed path backwards from it's current point to where it began.

- To track: follow the emerging path forwards from your starting point to wherever<br>
the thing currently is.